In [13]:
%load_ext autoreload
%autoreload 2

import wandb
from dotenv import dotenv_values
import pandas as pd

from src.utils import download_table
from src.models.analysis import delong_roc_test


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
our_model_preds = download_table("289402rh","test_predictions")
xgb_preds = download_table("2r7v1e2d","test_predictions")

In [20]:
our_model_preds["date"] = pd.to_datetime(our_model_preds["date"]).dt.date
xgb_preds["date"] = pd.to_datetime(xgb_preds["date"]).dt.date

In [21]:
xgb_preds[xgb_preds["label"]==1].sort_values("date")

,participant_id,date,label,pred
17016,625c578280a31eb620459ff3b2d508b5,2020-02-10,1.0,0.016943
94515,fc8197485fdd7526ab6fa27a9247885a,2020-02-10,1.0,0.982843
126996,e6f9522a10d8a9b4c4bf80e05fdca6f7,2020-02-10,1.0,0.005180
6884,88a38a101135225184b9fd46a7e29bb1,2020-02-10,1.0,0.013413
125018,0950145f691d8379fb7c90f1ceb4c529,2020-02-10,1.0,0.013519
...,...,...,...,...
130237,c50c9310c2bf285a3636294831fbef26,2020-03-30,1.0,0.009396
61924,6f6cd9f458f65bd30da0ac16ef83c02a,2020-03-31,1.0,0.011147
66900,14bbdca33aad9f88c9669aa227696d1e,2020-03-31,1.0,0.007734
59913,c8891eca71deb8603db56c0cb5d1e025,2020-03-31,1.0,0.014080


In [22]:
our_model_preds[our_model_preds["label"]==1].sort_values("date")

,participant_id,date,label,pred
59114,d7910e655fe05c9fa9569705f108a798,2020-02-16,1,0.018381
88855,aa3a4bfefee70d385c98c9d794f01690,2020-02-16,1,0.010757
25478,6f88d9150a67bc4da047b745c10855ff,2020-02-16,1,0.016719
73950,925eec4cd73cf714335ec64b6215ba3d,2020-02-16,1,0.510906
41679,70804415f9570c6abca054944ed4667a,2020-02-16,1,0.053115
...,...,...,...,...
16238,557d2ba26a933779664a43c00e7d7a28,2020-03-31,1,0.667339
18119,92762457b1a8dc31a1e381acea605b3d,2020-03-31,1,0.051493
92744,457a3a99dac4e8045441775840925137,2020-03-31,1,0.032813
29728,cc26c340e4e0525188f80a15587f0fcf,2020-03-31,1,0.008450


In [23]:
index = ["participant_id","date"]
merged = our_model_preds.set_index(index).join(xgb_preds.set_index(index),lsuffix="_ours",rsuffix="_xgb").dropna()

In [27]:
log_p = delong_roc_test(merged["label_ours"].values, merged["pred_ours"].values, merged["pred_xgb"].values)[0][0]
10**log_p

0.025609166696762945